In [1]:
import zipfile
import os

In [4]:
# Extract the zip file
# Create a directory for extraction
extract_dir = "/content/train"
os.makedirs(extract_dir, exist_ok=True)

# Open and extract the zip file
with zipfile.ZipFile("/content/train_SOaYf6m.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Contents extracted to {extract_dir}")

Contents extracted to /content/train


In [5]:
# Extract the zip file
# Create a directory for extraction
extract_dir = "/content/test"
os.makedirs(extract_dir, exist_ok=True)

# Open and extract the zip file
with zipfile.ZipFile("/content/test.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Contents extracted to {extract_dir}")

Contents extracted to /content/test


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:

# Define transformations for the training and testing data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet18 expects 224x224 input size
    transforms.ToTensor(),
])


In [4]:

# Load the training and testing datasets
train_data = datasets.ImageFolder('/content/train', transform=transform)
test_data = datasets.ImageFolder('/content/test/test', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [5]:
# Load the pre-trained ResNet18 model and modify the final layer for binary classification
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Adjust the final layer for binary classification


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [7]:
# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 0.01939422974241146
Epoch 2, Loss: 2.1400435282368955e-05
Epoch 3, Loss: 1.7628771508276947e-05
Epoch 4, Loss: 1.5460911182910077e-05
Epoch 5, Loss: 1.3818796498919648e-05
Epoch 6, Loss: 1.2390978394545823e-05
Epoch 7, Loss: 1.1217227770307423e-05
Epoch 8, Loss: 1.0147731073457917e-05
Epoch 9, Loss: 9.224876804010646e-06
Epoch 10, Loss: 8.418813533009253e-06


In [21]:
# Custom dataset for test images listed in test.csv
class TestDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.test_df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.test_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.test_df.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_name

# Load the test dataset from test.csv
test_data = TestDataset(csv_file='/content/test/test/test.csv', root_dir='/content/test/test/images', transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [26]:
# Testing the model and preparing the results for the excel sheet
# Testing the model and preparing the results for the excel sheet
model.eval()
results = []
with torch.no_grad():
    for images, img_names in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        for i in range(len(images)):
            image_name = img_names[i].split('/')[-1]  # Extract image name from path
            emergency_or_not = 1 if predicted[i].item() == 1 else 0
            results.append([image_name, emergency_or_not])

In [28]:
# Create a DataFrame and save to a csv file
df = pd.DataFrame(results, columns=['image_names', 'emergency_or_not'])
df.to_csv('emergency_vehicle_results.csv', index=False)

print("Results have been saved to emergency_vehicle_results.csv")

Results have been saved to emergency_vehicle_results.csv
